# Test Agent A2A Walkthrough

This notebook sends a JSON-RPC message to the Test Agent A2A endpoint and prints the response.

Set `APP_HOST` to your app host (without scheme) or to a full URL.

In [12]:
from __future__ import annotations

from uuid import uuid4

import httpx
from databricks.sdk import WorkspaceClient

APP_HOST = "localhost:8000"

if APP_HOST.startswith("http://") or APP_HOST.startswith("https://"):
    BASE_URL = APP_HOST.rstrip("/")
else:
    BASE_URL = f"http://{APP_HOST}".rstrip("/")

if "localhost" in APP_HOST:
    headers: dict[str, str] = {}
else:
    w = WorkspaceClient()
    headers = w.config.authenticate()

A2A_URL = f"{BASE_URL}/test-agent/"
print("A2A_URL:", A2A_URL)

A2A_URL: http://localhost:8000/test-agent/


In [13]:
def send_test_agent_message(text: str) -> dict:
    payload = {
        "jsonrpc": "2.0",
        "id": uuid4().hex,
        "method": "message/send",
        "params": {
            "message": {
                "role": "user",
                "parts": [{"kind": "text", "text": text}],
                "message_id": uuid4().hex,
            }
        },
    }
    resp = httpx.post(A2A_URL, json=payload, headers=headers, timeout=60.0)
    resp.raise_for_status()
    return resp.json()

In [14]:
response = send_test_agent_message("hello from notebook")
print(response)

{'id': '42dab285d49b43f090d1b914aa564a43', 'jsonrpc': '2.0', 'result': {'artifacts': [{'artifactId': '3ed9c620-e32e-4df1-819d-ff9452d98b4d', 'name': 'test_agent_response', 'parts': [{'kind': 'text', 'text': '{\n  "message": "hello world from Test Agent",\n  "handshake": {\n    "context_id": "e105f906-42f5-47bd-8372-3d8c3c058e47",\n    "task_id": "e87019dd-b2ff-487b-a184-2cb878de5218",\n    "message_id": "803e562c11244b31b827472520e437dc"\n  },\n  "request": {\n    "input_text": "hello from notebook"\n  },\n  "history_size": 1\n}'}]}], 'contextId': 'e105f906-42f5-47bd-8372-3d8c3c058e47', 'history': [{'contextId': 'e105f906-42f5-47bd-8372-3d8c3c058e47', 'kind': 'message', 'messageId': '803e562c11244b31b827472520e437dc', 'parts': [{'kind': 'text', 'text': 'hello from notebook'}], 'role': 'user', 'taskId': 'e87019dd-b2ff-487b-a184-2cb878de5218'}], 'id': 'e87019dd-b2ff-487b-a184-2cb878de5218', 'kind': 'task', 'status': {'state': 'completed', 'timestamp': '2026-02-09T05:42:51.367539+00:00'}}